# Machine Learning Base Line

## Initialization

In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql.types import *

conf = SparkConf().setAppName("preprocess").setMaster("local")
sc = SparkContext.getOrCreate(conf)
spark = SparkSession.builder.master("local").appName("preprocess").getOrCreate()

### Load and Split Data

In [3]:
from pyspark.mllib.util import Vectors, MLUtils
from pyspark.mllib.linalg import VectorUDT
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import DataType, StringType

def output_csv(df, path):
    udf = UserDefinedFunction(lambda x: Vectors.stringify(x), StringType())
    new_df = df.withColumn('features', udf(df.features))
    
    new_df.write.csv(path, header=True)
    
def read_csv(path):
    df = spark.read.csv(path, header=True, inferSchema=True)
    
    udf = UserDefinedFunction(lambda x: Vectors.parse(x), VectorUDT())
    # https://spark.apache.org/docs/latest/ml-migration-guides.html
    new_df = MLUtils.convertVectorColumnsToML(df.withColumn('features', udf(df.features)))
    
    return new_df

In [4]:
df = read_csv("./data/DATA_TFIDF_HADM_TOP10.csv")
df_train, df_test = df.randomSplit(weights=[0.75, 0.25], seed=12345)
df_train.cache()
df_test.cache()
print "Train:", df_train.count()
print "Test:", df_test.count()
df_train.show()
df_test.show()

Train: 30430
Test: 10132
+------+----+----+-----+----+-----+-----+-----+-----+----+----+--------------------+
|    id|4019|2724|25000|4280|41401|53081|51881|42731|5849|5990|            features|
+------+----+----+-----+----+-----+-----+-----+-----+----+----+--------------------+
|100050|   0|   0|    0|   1|    1|    1|    0|    1|   0|   1|(40000,[69,78,104...|
|100053|   0|   0|    1|   0|    0|    0|    0|    1|   0|   0|(40000,[794,2044,...|
|100059|   1|   0|    1|   0|    1|    0|    0|    0|   0|   0|(40000,[130,207,3...|
|100061|   0|   0|    0|   1|    0|    0|    0|    0|   1|   0|(40000,[24,151,20...|
|100065|   1|   0|    0|   0|    0|    0|    0|    0|   1|   0|(40000,[115,794,8...|
|100066|   1|   0|    0|   0|    1|    1|    0|    0|   0|   0|(40000,[130,184,1...|
|100074|   0|   0|    0|   0|    0|    0|    1|    0|   0|   0|(40000,[32,62,71,...|
|100077|   0|   1|    0|   0|    1|    0|    0|    0|   0|   0|(40000,[78,102,12...|
|100281|   0|   0|    0|   1|    1|    0

### Evaluator

In [5]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.types import StringType, IntegerType
import pyspark.sql.functions as F

concat_udf = F.udf(lambda cols: float(int("".join([str(int(x)) for x in cols]), 2)), DoubleType())

def evaluate(df, labelCols, metrics=["f1", "weightedPrecision", "weightedRecall", "accuracy"]):
    evaluator = MulticlassClassificationEvaluator()
    labelCols2 = [i+"_pred" for i in labelCols]
    df2 = df.withColumn("_label", concat_udf(F.array(labelCols)))
    df2 = df2.withColumn("_pred", concat_udf(F.array(labelCols2)))
    
    output = {}
    for m in metrics:
        result = evaluator.evaluate(df2, {evaluator.metricName: m,
                                         evaluator.predictionCol: "_pred",
                                         evaluator.labelCol: "_label"})
        output[m] = result
        
    return output
    

## Logistic Regression

Define our custom Logistic Regression class

In [5]:
from pyspark.ml.classification import LogisticRegression

class CustomLogisticRegression:
    def __init__(self):
        pass
    
    def fit(self, df, maxIter=100, regParam=0.0, featuresCol="features", ignoreCols=["id"]):
        self.featuresCol = featuresCol
        self.labelCols = df.columns
        self.labelCols.remove("features")
        for c in ignoreCols:
            self.labelCols.remove(c)
        self.models = []
        
        for c in self.labelCols:
            lr = LogisticRegression(featuresCol=featuresCol,
                                    labelCol=c,
                                    predictionCol=c+"_pred",
                                    probabilityCol=c+"_prob",
                                    rawPredictionCol=c+"_rpred",
                                    maxIter=maxIter,
                                    regParam=regParam,
                                    family="binomial")
            model = lr.fit(df)
            self.models.append(model)
            
    def predict(self, df):
        df_out = df
        for c, m in zip(self.labelCols, self.models):
            df_out = m.transform(df_out)
            
        return df_out
        
        

Evaluate with our data

In [6]:
clr = CustomLogisticRegression()
clr.fit(df_train)
df_pred_train = clr.predict(df_train)
df_pred_test = clr.predict(df_test)

In [ ]:
for maxIter in [5, 10, 25, 50, 75, 100]:
    clr = CustomLogisticRegression()
    clr.fit(df_train, maxIter=maxIter)
    df_pred_train = clr.predict(df_train)
    df_pred_test = clr.predict(df_test)

    print "maxIter: ", maxIter
    print evaluate(df_pred_train, clr.labelCols)
    print evaluate(df_pred_test, clr.labelCols)

maxIter:  5
{'weightedPrecision': 0.3191686583865087, 'f1': 0.2422791139786058, 'weightedRecall': 0.25681892868879336, 'accuracy': 0.25681892868879397}
{'weightedPrecision': 0.10625387820800844, 'f1': 0.09190509299831698, 'weightedRecall': 0.12258191867350976, 'accuracy': 0.12258191867350968}
maxIter:  10
{'weightedPrecision': 0.7841250725433033, 'f1': 0.7620669774918355, 'weightedRecall': 0.7511994742030924, 'accuracy': 0.7511994742030891}
{'weightedPrecision': 0.12330947229469667, 'f1': 0.11949211928027403, 'weightedRecall': 0.12425977102250312, 'accuracy': 0.12425977102250296}
maxIter:  25
{'weightedPrecision': 0.9999712454814376, 'f1': 0.9999675758571632, 'weightedRecall': 0.9999671376930708, 'accuracy': 0.999967137693066}
{'weightedPrecision': 0.10715632789997416, 'f1': 0.10053642873404066, 'weightedRecall': 0.09800631662060805, 'accuracy': 0.09800631662060798}
maxIter:  50
{'weightedPrecision': 1.0000000000000049, 'f1': 1.0000000000000049, 'weightedRecall': 1.0000000000000049, 'a

## Random Forest

Define our custom Logistic Regression class

In [6]:
from pyspark.ml.classification import RandomForestClassifier

class CustomRandomForestClassifier:
    def __init__(self):
        pass
    
    def fit(self, df, maxDepth=5, maxBins=32, numTrees=20, regParam=0.0, featuresCol="features", ignoreCols=["id"]):
        self.featuresCol = featuresCol
        self.labelCols = df.columns
        self.labelCols.remove("features")
        for c in ignoreCols:
            self.labelCols.remove(c)
        self.models = []
        
        for c in self.labelCols:
            lr = RandomForestClassifier(featuresCol=featuresCol,
                                        labelCol=c,
                                        predictionCol=c+"_pred",
                                        probabilityCol=c+"_prob",
                                        rawPredictionCol=c+"_rpred",
                                        maxDepth=maxDepth,
                                        maxBins=maxBins,
                                        impurity="gini",
                                        numTrees=numTrees,
                                        seed=None)
            model = lr.fit(df)
            self.models.append(model)
            
    def predict(self, df):
        df_out = df
        for c, m in zip(self.labelCols, self.models):
            df_out = m.transform(df_out)
            
        return df_out

Evaluate with our data

In [ ]:
for maxDepth in [5, 10, 25, 50, 75, 100]:
    clr = CustomRandomForestClassifier()
    clr.fit(df_train, maxDepth=maxDepth)
    df_pred_train = clr.predict(df_train)
    df_pred_test = clr.predict(df_test)

    print "maxDepth: ", maxDepth
    print evaluate(df_pred_train, clr.labelCols)
    print evaluate(df_pred_test, clr.labelCols)